# van Emde Boas Trees
| References |
|:----------:|
| [`MIT OCW Lecture`](https://www.youtube.com/watch?v=hmReJCupbNU) |
| [`Stanford Course Slides`](https://web.stanford.edu/class/archive/cs/cs166/cs166.1146/lectures/14/Small14.pdf) |

## Bit Vector

Vector of boolean values of a _fixed_ size. Contains min & max, but due to potential for delete, operations for getting min & max can run as long as the length of the array.

**u** : universe size  
**n** : number of elements in vector  

| Operation | Worst Case TC |
|:---------:|:---------------:|
| `Lookup` | O(1) |
| `Insert` | O(1) |
| `Delete` | O(1) |
| `Successor` | O(**u**) |
| `Predecessor` | O(**u**) |
| `Min` | O(**u**) |
| `Max` | O(**u**) |

`Memory` : **u**

In [ ]:
import numpy as np

In [1]:
class BitVector():
    def __init__(self, u, init_arr=None):
        self.u = u
        self.bt_vr = np.linspace(0, u, u, dtype=bool) #linspace stop is inclusive by default
        if init_arr:
            self.n = len(init_arr)
            for x in init_arr:
                self.insert(x)
        else:
            self.n = 0
            self._prev_minimum = None
            self._prev_maximum = None
        return
    
    def __str__(self): 
        print('[',self.bt_vr[0], end='')
        for i in range(1, len(self.bt_vr)-1):
            val = 1 if self.bt_vr[i] else 0
            print(f' {val} ', end='')
        print(self.bt_vr[-1],']')
        
    def lookup(self, x):
        self.check_x(x)
        return self.bt_vr[x]
    
    def check_x(self, x):
        if x > u:
            raise ValueError(f'Value: {x} is greater than universe: {u}')
        if type(x) not in (int, np.int):
            raise TypeError(f'type: {type(x)} not a valid integer')
        return
        
    def insert(self, x):
        self.check_x(x)
        self.bt_vr[x] = True
        if not self.n: # never set n to non integer or this could cause bugs
            self.minimum = x
            self.maximum = x
        elif self.minimum > x:
            self.minimum = x
        elif self.maximum < x:
            self.maximum = x
        self.n += 1
        return
        
    def delete(self, x):
        self.check_x(x)
        self.bt_vr[x] = False
        if self.minimum == x:
            # waits for next minimum get operation to perform min search
            self.minimum = None
        elif self.maximum == x:
            # waits for next maximum get operation to perform max search
            self.maximum = None 
        self.n -= 1
        return
    
    def successor(self, x):
        self.check_x(x)
        while not self.bt_vr[x]:
            x += 1
            if x > self.u:
                return None
        return x
        
    def predecessor(self, x):
        self.check_x(x)
        while not self.bt_vr[x]:
            x -= 1
            if x < 0:
                return None
        return x

    # numpy arrays (and python lists) have min & max operations, 
    # these are not asymptotically efficient under the same assumptions here
    @property
    def minimum(self, x):
        self.check_x(x)
        if self._minimum is None and self._prev_minimum is not None:
            self._minimum = self.successor(self._prev_minimum)
        elif self._minimum is None:
            # could handle initial case here, worse for debugging
            raise ValueError('Algorithm went bad, no reference minimums')
        return self._minimum
    
    @minimum.setter
    def minimum(self, x):
        self.check_x(x)
        if x is None: self._prev_minimum = self._minimum
        else: # for debugging only
            self._prev_minimum = None
        self._minimum = x
        
    @property
    def maximum(self, x):
        self.check_x(x)
        if self._maximum is None and self._prev_maximum is not None:
            self._maximum = self.predecessor(self._prev_maximum)
        elif self._maximum is None:
            raise ValueError('Algorithm went bad, no reference maximums')
        return self._maximum
        
    @maximum.setter
    def maximum(self, x):
        self.check_x(x)
        if x is None: self._prev_maximum = self._maximum
        else: # for debugging only
            self._prev_maximum = None
        self._maximum = x

## Summary Vector

Bit vector with single added layer that allows skipping unoccupied chunks of array. This is done by splitting the bit vector into "galaxies" (or clusters) of size sqrt(u).

**u** : universe size  
**n** : number of elements in vector  

| Operation | Worse Case TC |
|:---------:|:---------------:|
| `Lookup` | O(1) |
| `Insert`  | O(1) |
| `Delete`*  | O(**$ \sqrt{u} $**) |
| `Successor` | O(**$ \sqrt{u} $**) |
| `Predecessor` | O(**$ \sqrt{u} $**) |
| `Min` | O(**$ \sqrt{u} $**) |
| `Max` | O(**$ \sqrt{u} $**) |

`Memory` : **u** + **$ \sqrt{u} $**

*_Delete requires updating the summary vector efficiently, which is delayed for the moment_

In [ ]:
class SummaryVector(BitVector):
    def __init__(self, u, init_arr=None, n_galaxies=None):
        self.n_galaxies = u**.5 if n_galaxies is None else n_galaxies # sqrt(u)
        self.galaxy_size = u // self.n_galaxies
        self.sm_vr = np.linspace(0, self.n_galaxies, self.n_galaxies, dtype=bool)
        super().__init__(u, init_arr=init_arr)
        # setting n_galaxies by arg will likely decrease performance without information about input data
        # n_galaxies = galaxy_size iff n_galaxies == sqrt(u)
        
    def __str__(self):
        string = ''
        for val in self.sm_vr:
            temp_str = '['
            temp_str += ' '*((len(galaxy_size)-3) // 2)
            temp_str += str(val)
            temp_str += ' '*(len(galaxy_size) - len(temp_str))
            string += temp_str
            
        string += '\n[' + str(self.bt_vr[0])
        for i in range(1, len(self.bt_vr)-1):
            val = '1' if self.bt_vr[i] else '0'
            string += ' ' + val + ' '
        string += f'{self.bt_vr[-1]}]'
        return string
        
    def insert(self, x):
        self.sm_vr[x % self.galaxy_size] = True
        super().insert(x)
        
    def delete(self):
        raise NotImplementedError()

## Hashed Recursive Bit Vector

In [ ]:
# class BitVector():
#     def __init__(self, u, init_arr=None):
#         self.u = u
#         self.bt_vr = dict()
#         if init_arr:
#             self.n = len(init_arr)
#             for x in init_arr:
#                 self.insert(x)
#         else:
#             self.n = 0
#             self._prev_minimum = None
#             self._prev_maximum = None
#         return
    
#     def __str__(self): 
#         print('[',self.bt_vr[0], end='')
#         for i in range(1, self.u-1):
#             val = 1 if i in self.bt_vr else 0
#             print(f' {val} ', end='')
#         print(self.bt_vr[self.u],']')
        
#     def lookup(self, x):
#         self.check_x(x)
#         return True if x in self.bt_vr else False
    
#     def check_x(self, x):
#         if x > u:
#             raise ValueError(f'Value: {x} is greater than universe: {u}')
#         if type(x) not in (int, np.int):
#             raise TypeError(f'type: {type(x)} not a valid integer')
#         return
        
#     def insert(self, x):
#         self.check_x(x)
#         self.bt_vr[x] = True
#         if not self.n: # never set n to non integer or this could cause bugs
#             self.minimum = x
#             self.maximum = x
#         elif self.minimum > x:
#             self.minimum = x
#         elif self.maximum < x:
#             self.maximum = x
#         self.n += 1
#         return
        
#     def delete(self, x):
#         self.check_x(x)
#         del self.bt_vr[x]
#         if self.minimum == x:
#             # waits for next minimum get operation to perform min search
#             self.minimum = None
#         elif self.maximum == x:
#             # waits for next maximum get operation to perform max search
#             self.maximum = None 
#         self.n -= 1
#         return
    
#     def successor(self, x):
#         self.check_x(x)
#         while not x in self.bt_vr:
#             x += 1
#             if x > self.u:
#                 return None
#         return x
        
#     def predecessor(self, x):
#         self.check_x(x)
#         while not x in self.bt_vr:
#             x -= 1
#             if x < 0:
#                 return None
#         return x

#     @property
#     def is_empty(self):
#         if not self.n:
#             return True
#         else:
#             return False
        
#     # numpy arrays (and python lists) have min & max operations, 
#     # these are not asymptotically efficient under the same assumptions here
#     @property
#     def minimum(self, x):
#         self.check_x(x)
#         if self._minimum is None and self._prev_minimum is not None:
#             self._minimum = self.successor(self._prev_minimum)
#         elif self._minimum is None:
#             # could handle initial case here, worse for debugging
#             raise ValueError('Algorithm went bad, no reference minimums')
#         return self._minimum
    
#     @minimum.setter
#     def minimum(self, x):
#         self.check_x(x)
#         if x is None: self._prev_minimum = self._minimum
#         else: # for debugging only
#             self._prev_minimum = None
#         self._minimum = x
        
#     @property
#     def maximum(self, x):
#         self.check_x(x)
#         if self._maximum is None and self._prev_maximum is not None:
#             self._maximum = self.predecessor(self._prev_maximum)
#         elif self._maximum is None:
#             raise ValueError('Algorithm went bad, no reference maximums')
#         return self._maximum
        
#     @maximum.setter
#     def maximum(self, x):
#         self.check_x(x)
#         if x is None: self._prev_maximum = self._maximum
#         else: # for debugging only
#             self._prev_maximum = None
#         self._maximum = x

In [ ]:
class VEBTree():
    def __init__(self, u, init_arr=None):
        self.u = u
        self.n_galaxies = int(u**.5) # use perfect squares for convenience
        self.minimum = None
        self.maximum = None
#         self.n = 0 # not strictly necessary
        if init_arr:
            for val in init_arr:
                self.insert(val)
        return
    
    def high(self, x):
        high = x // self.n_galaxies
        return high
    
    def low(self, x):
        low = x % self.n_galaxies
        return low
    
    def index(self, x, y):
        index = x * self.n_galaxies + y
        
    def insert(self, x):
#         swap = lambda a, b : a, b = b, a
        def swap(a,b):
            a, b = b, a
            
        if self.minimum is None: # if node is empty
            self.minimum, self.maximum = x, x
            return
        elif x < self.minimum: # if x is the new minimum
            swap(x, self.minimum)
        elif x > self.maximum:
            swap(x, self.maximum) # if x is the new maximum
        if self.u > 2: # propagate down the tree if not at 'leaf'
            i = self.high(x)
            j = self.low(x)
            self.cluster[i].insert(j)
            if self.cluster[i].minimum is not None: # do not track yet if lower node is empty
                self.summary.insert(i)
        return
    
    def delete(self, x):
        if self.minimum == self.maximum: # if only one node
            self.minimum = None
            self.maximum = None
            return
#         elif self.u == 2: # 
#             self.min = 1 if not x else 0
#             self.max = self.min
#             return
#         if x == self.minimum:
#             i = self.summary.minimum
#             j = self.cluster[i].minimum
#             x = self.index(i, j)
#             self.minimum = x
#         self.cluster[self.high(x)].delete(self.low(x))
#         if self.cluster[self.high(x)].minimum is None:
#             self.summary.delete(self.high(x))
#         if x == self.maximum:
#             i = self.summary.maximum
#             if i is None:
#                 self.maximum = self.minimum
#             else:
#                 j = self.cluster[i].maximum
#                 self.maximum = self.index(i, j)
        return
    
    def successor(self, x):
        if self.u == 2:
            if not x and self.maximum == 1:
                return 1
            else:
                return None
        elif self.min is not None and x < self.min:
            return self.min
        else:
            i = self.high(x)
            j = self.low(x)
            if i in self.bucket:
                offset = self.bucket[i].get_successor